# Agenda, day 2

1. Recap and Q&A
2. Magic methods / "dunder" methods
3. Class attributes
4. Finding attributes with ICPO
5. Inheritance -- what it is, and how ICPO influences it
6. Three models for method inheritance
7. Data inheritance
8. What next?

# Recap

How do we solve problems? It's easiest if we can use a data structure that is appropriate for solving that problem. Objects allow us to create a data structure that is specifically designed to solve a problem. Even though we might be using ints, strings, lists, and dicts to implement our object, the fact that we can think about it at a higher level makes it easier to work with.

The whole idea of object-oriented programming is thus: Create new data structures, along with methods, that together make it easier to think about and use these structures and thus solve our problems.

1. Methods are defined as part of the class body. And actually, methods are attributes on the class. However, we invoke them via the instance, and Python does a little magical substitution, turning the instance into the first argument passed to the method.
2. The first parameter in every method is traditionally called `self`, and gets that instance assigned to it.
3. If you try to invoke a method that doesn't exist, you'll get an "attribute error," with Python telling you that the method doesn't exist.
4. We define new "classes," factories for our data structures. We start the class definition with the reserved word `class`, then the name we want to give it, and then any number of methods that we might want to invoke on our data.
5. If we have a `Person` class, then it creates *instances* of `Person`. Every object has a `type`, and we can invoke `type()` on it to find out its class. In this way, `class` and `type` are two ways to say the same thing.
6. The most important method in a class is `__init__`, the initializer. (We pronounce it "dunder init," because it has a double underscore before and after the word "init.") Its job is to add attributes to a new object after that object is created, but before it's returned to the caller. Typically, we'll assign many attributes based on parameters, but we can add fewer or more, depending on what we want to do. In theory, you can add attributes to an object whenever you want... but it's a good idea to only add new attributes in `__init__`, for easier maintenance.
7. Each time we invoke a method on an instance, the first argument is the instance itself, and is assigned to `self`. You can use another word instead of `self`, and if you forget to make `self` the first parameter, then whatever is the first parameter will get the instance assigned to it. But you really should use `self`.
8. Python doesn't have protected or private status for attributes. (Other languages do, and they call attributes "instance variables" when they're set on `self`.) Whereas other languages have getters and setters so that people don't access those values directly, Python *encourages* us to use them directly!
   

In [1]:
class Person:
    def __init__(self, name, shoe_size):
        self.name = name
        self.shoe_size = shoe_size
        self.bank_balance = 0   # attribute that I set, with a default/starter value, without any connection to a parameter

    def greet(self):
        return f'Hello, {self.name}!'  # if I just say "name", and not "self.name", then Python will not find my attribute

p = Person('Reuven', 46)    

In [2]:
p.greet()   # the same as Person.greet(p)

'Hello, Reuven!'

In [3]:
Person.greet(p)

'Hello, Reuven!'

In [4]:
p.name

'Reuven'

In [5]:
p.name = 'whatever'

In [6]:
p.name

'whatever'

In [7]:
p.greet()

'Hello, whatever!'

In [8]:
p.bank_balance

0

# Exercise: Cellphone

1. Define a `Cellphone` class. Each instance will have two attributes:
    - `number`
    - `model`
2. You should be able to invoke the `call` method on an instance of `Cellphone. It'll return a string saying, "Calling" and then print the number that is calling. You'll have to provide an argument, the number to call.

In [14]:
class Cellphone:
    def __init__(self, number, model):
        self.number = number
        self.model = model

    def call(self, other_number):
        return f'Phone {self.number} is calling {other_number}'

c1 = Cellphone(12345, 'iPhone')
c2 = Cellphone(67890, 'Android')

c1.call(2468)

'Phone 12345 is calling 2468'

In [15]:
c2.call(c1.number)  

'Phone 67890 is calling 12345'

In [10]:
type(c1)

__main__.Cellphone

In [11]:
vars(c1)

{'number': 12345, 'model': 'iPhone'}

In [12]:
type(c2)

__main__.Cellphone

In [13]:
vars(c2)

{'number': 67890, 'model': 'Android'}

In [16]:
# what happens if I print my cellphones?

print(c1)

In [17]:
print(c2)

# Magic methods

When we perform certain operations in Python, behind the scenes, the operator is translated into a method call. For example, when we compare values with `==`, there's really a method that is being invoked.

Every operator that you can think of, plus many you probably cannot, are all implemented using methods.

The thing is, these methods don't always need to be defined. And you probably shoudn't be invoking them on your own very often. Plus, if we define these methods, then we really change the way that our objects work, because Python sees the method and uses it .

For all of these reasons, these special methods, known as "magic methods," have special names. All of their names are "dunders," meaning that they start and end with a double underscore, `__`. (That's not `_`, but rather `__` both before and after the method name.)

I want to show you some magic methods now, and we'll use them in a number of classes.

It's typical to implement a number of these methods. Also, new Python functionality is generally implemented by adding new dunder methods.

You should *not* name a method with dunders unless you know what you're doing! You might get yourself into trouble, either now or down the road if/when Python uses that name.

# Simple magic method: `__len__`

When you call `len` (the function) on an object, it in turn looks for the `__len__` method on that object.



In [18]:
len('abcd')

4

In [19]:
len(1234)

TypeError: object of type 'int' has no len()

In [20]:
class Person:
    def __init__(self, name):
        self.name = name

p = Person('Reuven')

len(p)

TypeError: object of type 'Person' has no len()

In [21]:
# I'm going to add a __len__ method that returns the length of the name

class Person:
    def __init__(self, name):
        self.name = name
    def __len__(self):
        return len(self.name)

p = Person('Reuven')

len(p)

6

# What happened?

1. I invoked `len(p)`
2. This was translated into `p.__len__()` by Python
3. Because `__len__` is defined on `Person`, it ran and its return value was returned to the caller.

# Exercise: How many scoops?

1. Yesterday, we defined both `Scoop` and `Bowl`. Any instance of `Bowl` has a `scoops` attribute, a list of `Scoop` instances.
2. Add the capability to invoke `len` on an instance of `Bowl`. That'll return the number of scoops in the bowl.
3. After doing so, test that it works -- call `len` on a `Bowl`, then add a new `Scoop`, then measure again.

In [24]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
    
class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def add_scoops(self, *new_scoops):
        for one_scoop in new_scoops:    # iterate over the list of scoops
            self.add_scoop(one_scoop)   # invoke self.add_scoop for each one, one at a time

    def flavors(self):
        output = []

        for one_scoop in self.scoops:
            output.append(one_scoop.flavor)

        return output

    def __len__(self):
        return len(self.scoops)

b = Bowl()   
b.add_scoops(s1, s2, s3)   # removed square brackets, so we're invoking it with 3 arguments
list_of_flavors = b.flavors()
print(list_of_flavors)     # print forwards

print(len(b))

['chocolate', 'vanilla', 'coffee']
3


In [25]:
s = 'abcde'

In [26]:
s.__len__()  # don't do this, but *can* you?

5

# Warning:

Make sure that you spell the magic methods correctly!

I've often seen people accidentally write `__int__` instead of `__init__`. That is the magic method that is invoked when you want to turn a value into an integer, when you invoke `int()` on it! 

# Next up:

1. Methods for `__str__` and `__repr__`
2. A handful of other magic methods, as well

# Printing our objects

If we try to print our cellphone or scoop or bowl objects, we find that they are super ugly.

In [27]:
print(p)  # Person

In [28]:
print(c1)

In [29]:
print(c2)

What's happening here?

We can actually use `print` on any object in Python. That's because `print` invokes `str` on whatever it's going to print. And everything in Python knows how to turn itself into a string.

How does the object know how to turn itself into a string? Because there is a default implementation of the `__str__` method that is shared by all objects. If we do nothing, then every object that doesn't have a special way to display itself will use this ugly way.

But this also explains what we can do to change things: If we implement `__str__` on our object, then our method will be invoked instead of the default, and then we can customize what string is returned. Note that `__str__` *must* return a string! But the string you do return can be anything at all.

In [33]:
class Person:
    def __init__(self, name):
        self.name = name
    def __len__(self):
        return len(self.name)
    def __str__(self):
        return f'Person with name = {self.name}!'

p = Person('Reuven')
print(p)  # -> print(str(p)) -> print(p.__str__())

Person with name = Reuven!


In [34]:
str(p)  # invoke str on p, what do I get back?

'Person with name = Reuven!'

# Exercise: Printable scoops

1. Modify the `Scoop` class such that printing a scoop gives us a string, "Scoop of FLAVOR".
2. Make sure that this works by printing a number of scoops.


In [36]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __str__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
    
class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def add_scoops(self, *new_scoops):
        for one_scoop in new_scoops:    # iterate over the list of scoops
            self.add_scoop(one_scoop)   # invoke self.add_scoop for each one, one at a time

    def flavors(self):
        output = []

        for one_scoop in self.scoops:
            output.append(one_scoop.flavor)

        return output

    def __len__(self):
        return len(self.scoops)

b = Bowl()   
b.add_scoops(s1, s2, s3)   # removed square brackets, so we're invoking it with 3 arguments
list_of_flavors = b.flavors()
print(list_of_flavors)     # print forwards

print(len(b))

print(s1)  
print(s2)
print(s3)

['chocolate', 'vanilla', 'coffee']
3
Scoop of chocolate
Scoop of vanilla
Scoop of coffee


# Exercise: Printable *bowls*

1. Now I want you to make it possible to say `print(b)`, where `b` is an instance of `Bowl`. Doing so should display "Bowl with:" and then each of the scoops on a line by itself.
2. (If you can number the scoops, even better, but don't worry about it too much.)
3. The printout for each of the scoops can be the result of invoking `str(a_scoop)`.

In [43]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __str__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
    
class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def add_scoops(self, *new_scoops):
        for one_scoop in new_scoops:    # iterate over the list of scoops
            self.add_scoop(one_scoop)   # invoke self.add_scoop for each one, one at a time

    def flavors(self):
        output = []

        for one_scoop in self.scoops:
            output.append(one_scoop.flavor)

        return output

    def __len__(self):
        return len(self.scoops)

    def __str__(self):
        output = 'Bowl of: \n'

        for index, one_scoop in enumerate(self.scoops, 1):
            output += f'\t{index}: {one_scoop}\n'
        
        return output

b = Bowl()   
b.add_scoops(s1, s2, s3)   # removed square brackets, so we're invoking it with 3 arguments
list_of_flavors = b.flavors()

print(list_of_flavors)     # print forwards

print(len(b))

print(s1)  
print(s2)
print(s3)

print('*** Now printing the bowl ***')
print(b)  

['chocolate', 'vanilla', 'coffee']
3
Scoop of chocolate
Scoop of vanilla
Scoop of coffee
*** Now printing the bowl ***
Bowl of: 
	1: Scoop of chocolate
	2: Scoop of vanilla
	3: Scoop of coffee

